<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Load-GPS" data-toc-modified-id="Load-GPS-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>Load GPS</a></span></li></ul></li></ul></li><li><span><a href="#Load-BRT-Original-Realized-Trips" data-toc-modified-id="Load-BRT-Original-Realized-Trips-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load BRT Original Realized Trips</a></span></li></ul></div>

In [2]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path 
current_path = Path().resolve()
abs_path = str(current_path.parent)
sys.path.append(abs_path)

import basedosdados as bd
import pymove as pm
import gtfs_kit as gk
import pandas as pd
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
import geopandas as gpd
import pyproj

import datetime
import plotly.express as px

from pymove.utils import log

In [3]:
log.set_verbosity('ERROR')

In [4]:
from rGTFS import simple, io, utils
from rGTFS.simple import *

In [142]:
gps_path = '../data/raw/gps_brt-2021-04-07.csv'
gtfs_path = '../data/output/gtfs_brt.zip'
rgtfs_path = '../data/output/rgtfs_brt.zip'
brt_gtfs_path = '../data/output/brt_gtfs.zip'

### Load GPS

In [351]:
gps = bd.read_sql(
    """
    SELECT latitude, longitude, timestamp_captura as datetime, linha as route_id, codigo as vehicle_id
    FROM `rj-smtr.dashboard_monitoramento_brt.registros_filtrada` 
    WHERE data = DATE '2021-04-07'
    AND placa is not null
    """,
    billing_project_id='rj-smtr'
)

Downloading: 100%|██████████| 117729/117729 [00:05<00:00, 22551.03rows/s]


In [359]:
gps.to_csv(gps_path, index=False)

In [173]:
realized_trips, rgtfs = simple.main(gtfs_path, gps_path, rgtfs_path)

In [165]:
gtfs = io.read_gtfs(gtfs_path, "km")

In [166]:
rgtfs = io.read_gtfs(rgtfs_path, "km")

In [167]:
brt_gtfs = io.read_gtfs(brt_gtfs_path, "km")

In [168]:
def hourly_trips(feed, label):
    f = deepcopy(feed.frequencies)
    f['n_trips'] = 3600 / f['headway_secs'] 
    f = f.groupby(['start_time'])['n_trips'].sum().reset_index()
    f['label'] = label
    return f

In [171]:
r = hourly_trips(rgtfs, 'Executado (GPS)')
g = hourly_trips(gtfs, 'Planejado (GTFS)')
b = hourly_trips(brt_gtfs, 'Executado (BRT)')
fig = px.line(pd.concat([r,g,b]), x='start_time', y='n_trips', color='label')

fig.update_layout(
    template='simple_white',
    yaxis_title='Número de Viagens',
    title='Número de Viagens BRT em 2021-04-07 (Planejado vs. Executado)',
    hovermode="x"
)

fig.show()

In [675]:
gtfs.compute_feed_time_series(trip_stats, ['20210407'], freq='6H')

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2021-04-07 00:00:00,10,31,31,605.349089,43.916667,13.784040
2021-04-07 06:00:00,29,8,29,318.219731,22.000000,14.464533
2021-04-07 12:00:00,0,0,0,0.000000,0.000000,0.000000
2021-04-07 18:00:00,0,0,0,0.000000,0.000000,0.000000


# Load BRT Original Realized Trips

In [ ]:
    ready_brt = ready_brt[
        (ready_brt['departure_datetime'] > '2021-04-07') &
        (ready_brt['departure_datetime'] < '2021-04-08')
             ]

    trips_to_gtfs(ready_brt, gtfs).write(brt_gtfs_path)